In [39]:
import pandas as pd
import numpy as np
import datetime as dt

In [40]:
# Reading in data
data = pd.read_csv('technicals.csv')

In [41]:
# Setting index to datetime and removing bad columns
data = data.set_index('timestamp').drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1).copy()

In [42]:
# Creating buy and sell signals
data['buy'] = np.where((data['low'] <= data['lower_band']) & (data['time'] > '10:30:00'), True, False)
data['sell'] = np.where((data['high'] >= data['ema_10']) & (data['time'] > "10:30:00"), True, False)

In [43]:
# Removing times when buy and sell are at the same time and isolating times when buy or sell
data = data[(data['buy'] & data['sell']) != True].copy()
data = data[(data['buy'] | data['sell']) == True].copy()

In [44]:
# Initializing last action variable
last = None

# Looping through each row
for index, row in data.iterrows():
    # Checking value of last and swithcing value accordingly 
    if row['sell'] == True and last != 'sell':
        # Sell action and last action was buy
        last = 'sell'
    elif row['buy'] == True and last != 'buy':
        # Buy action and last action was sell
        last = 'buy'
    else:
        # Consecutive
        data.at[index, 'buy'] = False
        data.at[index, 'sell'] = False

In [ ]:
# Removing first sell indicatoe
data = data.tail(data.shape[0] - 20)
data = data.copy()

In [54]:
data.to_csv('signals.csv')